## Overview

This notebook is about idenifying Instagram comments that are questions or contain certain key words indicating desire. 

### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import re
import unicodedata as ud
import string 
from nltk.corpus import stopwords
import nltk

%matplotlib inline

### Data

In [2]:
df = pd.read_csv('loreal_comments.csv')
full_commenters_df = pd.read_csv('loreal_commenters.csv')

In [4]:
full_commenters_df['username'] = full_commenters_df.instagram_username

### Helpers

In [5]:
def is_username(text):
    if re.match("@[A-Za-z0-9_.]+",text):
        return True
    return False

def is_hashtag(text):
    if re.match("#[A-Za-z0-9_.]+",text):
        return True
    return False

def is_hashtag(text):
    if re.match("#[A-Za-z0-9_.]+",text):
        return True
    return False

## regular expression for finding unicode
unicode_re = re.compile(u'['
    u'\U0001F300-\U0001F64F'
    u'\U0001F680-\U0001F6FF'
    u'\u2600-\u26FF\u2700-\u27BF]+', 
    re.UNICODE)

def strip_unicode(text):
    return unicode_re.sub('',text)

target_usernames = ['lorealmakeup']

def cleaned_text_for_questions(word_list):
    new_text = ' '.join([word.lower().strip('#') for word in word_list 
                         if not is_username(word)
                         and not word in set(target_usernames)])
    return strip_unicode(new_text)

translator = str.maketrans('', '', string.punctuation)

##### Helpers to Identify Questions

In [22]:
def contains_question_mark(text):
    if text.find('?') >= 0:
        return True
    return False

question_words = ['does','has','can','shall','is','was','where','why',
                  'who','what','which','how','are','when','which','whose',
                  'whom','please']
def starts_with_question_word(text):
    first_word = text.split(' ')[0]
    return first_word in set(question_words)

prepostions = ['with','for','about','from']
def ends_with_prep(text):
    text = text.translate(translator)
    last_word = text.split(' ')[-1]
    return last_word in set(prepostions)

def is_question(text):
    score = 0
    if contains_question_mark(text): score += 1 
    if starts_with_question_word(text): score += 1 
    if ends_with_prep(text): score += 1 
    if score > 0:
        return True
    return False

In [7]:
df['is_question'] = df['text'].apply(lambda x: is_question(cleaned_text_for_questions(x.split(' '))))

### Comments indicating desire

In [8]:
## Examples
# "Wow want it ❤"
# "Love this shade! 😍 #want"
# "Love that shade!! Gonna have to buy one💄"
# "You need this colour! @lucie.mccorkell"
# "I need this shade in my life 😍"
# "Desperately need this in America!!"

In [26]:
## Potential words to add:
desire_key_words = ['please','want','need','buy']
## negs
## 'check out my', 'check my', 'follow me'

In [25]:
def has_key_word(text):
    desire_key_words = set(['please','want','need','buy'])
    text = text.translate(translator)
    comment_words = set(text.split())
    if len(comment_words & desire_key_words) > 0:
        return True
    return False

In [20]:
def is_lead(text):
    return has_key_word(text)

In [12]:
df['is_lead'] = df['text'].apply(lambda x: is_lead(cleaned_text_for_questions(x.split(' '))))